# Imports

For the sake of simplicity, and to help the reader focus on what's needed in each section, imports won't be grouped on top but detailed as soon as they appear to be useful.

# Introduction

Within this notebook, we'll go further with a more complicated "commercial-type" database.

This will allow us to:
- manipulate more complex data types, and see how it can be handled in SQLAlchemy,
- perform more complex queries, both in plain SQL or with the ORM, these ways being implemented in SQLAlchemy,
- discover how to populate this kind of database thanks to the `Faker` library.

# Models

## Imports

In [1]:
from datetime import datetime

from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, ForeignKey, Table
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

## Definition

In [2]:
# Note that auto-increment is the default behavior for integer primary keys

Base = declarative_base()

class Client(Base):
    """A basic object representing `clients` table"""
    __tablename__ = "clients"
    # Fields
    id = Column(Integer, primary_key=True)
    name = Column(String)
    email = Column(String)
    # Relationship to orders (one to many)
    # One client may have many orders
    orders = relationship("Order", back_populates="client")
    

class Product(Base):
    """A basic object representing `products` table"""
    __tablename__ = "products"
    # Fields
    id = Column(Integer, primary_key=True)
    name = Column(String)
    price = Column(Float)
    # Relationship to order_details (one to many)
    # One product may have many order_details
    order_details = relationship("OrderDetail", back_populates="product")
    
    
class Order(Base):
    """A basic object representing `orders` table"""
    __tablename__ = "orders"
    # Fields
    id = Column(Integer, primary_key=True)
    client_id = Column(Integer, ForeignKey("clients.id"))
    date = Column(DateTime, default=datetime.utcnow)
    # Relationships
    # With clients (many to one): one client may have many orders
    client = relationship("Client", back_populates="orders")
    # With order_details (one to many): one order may have many order_details
    order_details = relationship("OrderDetail", back_populates="order")
    

class OrderDetail(Base):
    """An object figuring an association table between `orders` and `products`"""
    __tablename__ = "order_details"
    # Fields
    id = Column(Integer, primary_key=True)
    order_id = Column(Integer, ForeignKey("orders.id"))
    product_id = Column(Integer, ForeignKey("products.id"))
    quantity = Column(Integer)
    # Relationships
    # With order (many to one): one order may have many order_details
    order = relationship("Order", back_populates="order_details")
    # With product (many to one): one product may have many order_details
    product = relationship("Product", back_populates="order_details")

## Creation

In [3]:
# Create an engine that stores the data in the local directory's commercial.db
engine = create_engine('sqlite:///commercial.db', echo=True)

In [4]:
# Create all tables in the database which are defined by Base's subclasses
Base.metadata.create_all(engine)

2024-05-12 17:15:38,896 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-12 17:15:38,899 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("clients")
2024-05-12 17:15:38,900 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-12 17:15:38,901 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("clients")
2024-05-12 17:15:38,901 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-12 17:15:38,902 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2024-05-12 17:15:38,903 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-12 17:15:38,903 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("products")
2024-05-12 17:15:38,904 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-12 17:15:38,905 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2024-05-12 17:15:38,906 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-12 17:15:38,907 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("orders")
2024-05-12 17:15:38,908 INFO sqlalchemy.engine.Engine [raw sql] ()

# Populate the Database

In [5]:
from faker import Faker


fake = Faker()

## Define Data Requirements and Relationships

You need to understand the relationships:
- **Clients** can have multiple **Orders**.
- **Orders** can contain multiple **OrderDetails**.
- Each **OrderDetail** refers to one **Product**.
- **Products** can be associated with multiple **OrderDetails**.

## Generate Data for Independent Tables

You have to start with tables that do not have foreign key dependencies or where foreign keys refer to external data sources that do not require prior existence of other records in the database.

### Products

You'll begin with creating fake products first because they don't depend on any other data

In [6]:
products = [
    Product(name=fake.word(), price=fake.random_number(digits=2))
    for _ in range(100)
]